In [34]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

In [35]:
atomics = dict([(atomic.__name__, atomic) for atomic in mySimulator.atomic_registry.discovered_atomics])
Server = atomics['Server']
Dispatcher = atomics['Dispatcher']
Attacker = atomics['Attacker']
ServerQueue = atomics['ServerQueue']

In [36]:
number_of_servers = 5


dispatcherDefaultConfig = {
    'numberOfServers': number_of_servers
}
for i in range(number_of_servers):
    dispatcherDefaultConfig['server' + str(i)] = 'free'


serverDefaultConfig = {
    'distribution': 'exponential',
    'mean': 0.001,
    'setupTime': '00:00:10:00'
}

serverOnDefaultConifg = {
    'initialStatus': 'free',
    **serverDefaultConfig
}

serverOffDefaultConifg = {
    'initialStatus': 'off',
    **serverDefaultConfig
}

serverQueueDefaultConfig = {
    'size': 1000,
    'currentSizeFrequency': '00:00:00:10'
}

attackerDefaultConfig = {
    'file': 'attack-data/diffsShort.txt'
}

In [37]:
from pringles.models.errors import PortNotFoundException

dispatcher = Dispatcher('dispatcher', **dispatcherDefaultConfig)
queue = ServerQueue('queue', **serverQueueDefaultConfig)
servers = {}
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    servers[i] = Server(server_name, **serverOnDefaultConifg)
    try:
        dispatcher.get_port(server_name)
    except PortNotFoundException:
        dispatcher.add_outport(server_name)

attacker = Attacker('attacker', **attackerDefaultConfig)

In [38]:
print("--Dispatcher ports")
print("Inport names: ", [port.name for port in dispatcher.inports])
print("Outport names: ", [port.name for port in dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in servers[0].inports])
print("Outport names: ", [port.name for port in servers[0].outports])

print("--ServerQueue ports--")
print("Inport names: ", [port.name for port in queue.inports])
print("Outport names: ", [port.name for port in queue.outports])

print("--Attacker ports--")
print("Inport names: ", [port.name for port in attacker.inports])
print("Outport names: ", [port.name for port in attacker.outports])

--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStatus']
Outport names:  ['requestJob', 'server0', 'server1', 'server2', 'server3', 'server4']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']
--ServerQueue ports--
Inport names:  ['in', 'emit']
Outport names:  ['out', 'discarded', 'queueLoad', 'loadAvg']
--Attacker ports--
Inport names:  []
Outport names:  ['attack']


In [39]:
from pringles.models import Coupled

subcomponents = [dispatcher, queue, attacker] + list(servers.values())
    
top_model = Coupled(name='top', subcomponents=subcomponents)

# adding top inports
top_model.add_inport("serverStatus")
top_model.add_inport("queueLoad")
top_model.add_outport('discarded')
top_model.add_outport('jobDone')

# # adding serverStatus coupling to inport of dispatcher
top_model.add_coupling('serverStatus', dispatcher.get_port("serverStatus"))

# adding couplings between queue and dispatcher
top_model.add_coupling(queue.get_port("out"), dispatcher.get_port("newJob"))
top_model.add_coupling(dispatcher.get_port("requestJob"), queue.get_port("emit"))

# adding coupling between attacker and queue
top_model.add_coupling(attacker.get_port('attack'), queue.get_port('in'))
                                                                   
# adding couplings between dispatcher and servers
for i in range(number_of_servers):
    server_name = 'server' + str(i)
    top_model.add_coupling(dispatcher.get_port(server_name), servers[i].get_port('job'))
    top_model.add_coupling(servers[i].get_port('done'), dispatcher.get_port('jobDone'))
    top_model.add_coupling(servers[i].get_port('done'), 'jobDone')

# adding coupling between queue discarded port and top
top_model.add_coupling(queue.get_port('discarded'), 'discarded')

top_model

In [40]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(10),
                          working_dir='sim_results/queue-server-dispatcher-attacker'
                         )

results = mySimulator.run_simulation(a_simulation)

In [41]:
print(results.get_process_output())

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from sim_results/queue-server-dispatcher-attacker/2019-11-16-204236-f602aafad3d34ee9b89f2ed444d1156d/top_model
Loading events from 
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: sim_results/queue-server-di

In [42]:
# print(results.get_process_error_output())

In [43]:
display(results.output_df.head(100))

,time,port,value
0,00:00:00:000,jobdone,3.0
1,00:00:00:000,jobdone,5.0
2,00:00:00:000,jobdone,0.0
3,00:00:00:000,jobdone,7.0
4,00:00:00:000,jobdone,8.0
5,00:00:00:000,jobdone,6.0
6,00:00:00:001,jobdone,10.0
7,00:00:00:001,jobdone,2.0
8,00:00:00:001,jobdone,1.0
9,00:00:00:001,jobdone,11.0


In [44]:
print(results.logs_dfs.keys(),'\n\n')
display(results.logs_dfs['ParallelRoot'].head())

dict_keys(['server4', 'server3', 'server2', 'server1', 'server0', 'attacker', 'queue', 'dispatcher', 'top', 'ParallelRoot']) 




,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,Y,00:00:00:000,top(09),jobdone,3.0,ParallelRoot(00)
1,0,L,Y,00:00:00:000,top(09),jobdone,5.0,ParallelRoot(00)
2,0,L,Y,00:00:00:000,top(09),jobdone,0.0,ParallelRoot(00)
3,0,L,Y,00:00:00:000,top(09),jobdone,7.0,ParallelRoot(00)
4,0,L,Y,00:00:00:000,top(09),jobdone,8.0,ParallelRoot(00)


In [45]:
# Veamos que le llegan al server 0 los primneros 2 jobs, hasta que lo apagan en segundo 40
display(results.logs_dfs['server0'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(09),job,0.0,server0(04)
1,0,L,X,00:00:00:000,top(09),job,7.0,server0(04)
2,0,L,X,00:00:00:000,top(09),job,8.0,server0(04)
3,0,L,X,00:00:00:000,top(09),job,9.0,server0(04)
4,0,L,X,00:00:00:002,top(09),job,16.0,server0(04)
5,0,L,X,00:00:00:003,top(09),job,23.0,server0(04)
6,0,L,X,00:00:00:004,top(09),job,25.0,server0(04)
7,0,L,X,00:00:00:007,top(09),job,35.0,server0(04)
8,0,L,X,00:00:00:011,top(09),job,43.0,server0(04)
9,0,L,X,00:00:00:011,top(09),job,44.0,server0(04)


In [46]:
# Veamos que el siguiente job le llega al server 1 en el segundo 42, y luego lo apagan en el segundo 43
# Y se espera a que termine para apagarlo
display(results.logs_dfs['server1'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(09),job,1.0,server1(05)
1,0,L,X,00:00:00:001,top(09),job,13.0,server1(05)
2,0,L,X,00:00:00:004,top(09),job,24.0,server1(05)
3,0,L,X,00:00:00:006,top(09),job,31.0,server1(05)
4,0,L,X,00:00:00:007,top(09),job,36.0,server1(05)
5,0,L,X,00:00:00:010,top(09),job,41.0,server1(05)
6,0,L,X,00:00:00:011,top(09),job,46.0,server1(05)
7,0,L,X,00:00:00:011,top(09),job,48.0,server1(05)
8,0,L,X,00:00:00:012,top(09),job,50.0,server1(05)
9,0,L,X,00:00:00:012,top(09),job,51.0,server1(05)


In [47]:
# Por último cuando el 0 y el 1 están apagados, cuando llega el último job en el segundo 44, se lo envía al server 2
# que lo termina (ver logs dispatcher)
display(results.logs_dfs['server2'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(09),job,2.0,server2(06)
1,0,L,X,00:00:00:001,top(09),job,12.0,server2(06)
2,0,L,X,00:00:00:003,top(09),job,20.0,server2(06)
3,0,L,X,00:00:00:003,top(09),job,22.0,server2(06)
4,0,L,X,00:00:00:004,top(09),job,26.0,server2(06)
5,0,L,X,00:00:00:005,top(09),job,28.0,server2(06)
6,0,L,X,00:00:00:005,top(09),job,30.0,server2(06)
7,0,L,X,00:00:00:007,top(09),job,34.0,server2(06)
8,0,L,X,00:00:00:008,top(09),job,39.0,server2(06)
9,0,L,X,00:00:00:011,top(09),job,49.0,server2(06)


In [48]:
display(results.logs_dfs['dispatcher'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
1,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
2,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
3,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
4,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
5,0,L,X,00:00:00:000,top(09),jobdone,3.0,dispatcher(01)
6,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
7,0,L,X,00:00:00:000,top(09),jobdone,5.0,dispatcher(01)
8,0,L,X,00:00:00:000,top(09),newjob,1.0,dispatcher(01)
9,0,L,X,00:00:00:000,top(09),jobdone,0.0,dispatcher(01)
